In [2]:
import geopandas as gpd
import pandas as pd

In [3]:
cnts = gpd.read_file(r"L:\NaturalResources\Wetlands\Local Wetland Inventory\WAPO\EPA_2022_Tasks\Task 1 WD Mapping\GIS\Oregon_Counties.shp")

In [5]:
cnts.crs

<Geographic 2D CRS: EPSG:4269>
Name: NAD83
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: North America - onshore and offshore: Canada - Alberta; British Columbia; Manitoba; New Brunswick; Newfoundland and Labrador; Northwest Territories; Nova Scotia; Nunavut; Ontario; Prince Edward Island; Quebec; Saskatchewan; Yukon. Puerto Rico. United States (USA) - Alabama; Alaska; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Hawaii; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming. US Virgin Islands. British Virgin Islands

In [8]:
def get_point_from_lonlat(lon, lat, transprj=True, export=True):
    """
    Get a point from a lon/lat pair
    """
    df = pd.DataFrame([[lon, lat]], columns=['Longitude', 'Latitude'])
    gdf = gpd.GeoDataFrame(df, crs="EPSG:4326", geometry=gpd.points_from_xy(df.Longitude, df.Latitude))
    if transprj:
        gdf = gdf.to_crs(epsg=2992)
    if export:
        gdf.to_file(inpath + '\\test\point.shp')
    return gdf

In [11]:
pnt = get_point_from_lonlat(lon=-123.351673, lat=45.304685, transprj=False, export=False)

In [14]:
cnts.head(3)

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,STATE_NAME,LSAD,ALAND,AWATER,Shape_Leng,Shape_Area,geometry
0,41,041,01135856,0500000US41041,41041,Lincoln,Lincoln County,OR,Oregon,06,2.540731e+09,553147948.0,2.633880,0.291816,"POLYGON ((-124.11401 44.28984, -124.11334 44.2..."
1,41,047,01135859,0500000US41047,41047,Marion,Marion County,OR,Oregon,06,3.059028e+09,26557042.0,4.803694,0.351539,"POLYGON ((-123.18091 44.85755, -123.18053 44.8..."
2,41,055,01135863,0500000US41055,41055,Sherman,Sherman County,OR,Oregon,06,2.133152e+09,19617472.0,3.413503,0.247403,"POLYGON ((-121.03120 45.28586, -121.02635 45.2..."


In [19]:
polygon = cnts

In [24]:
point = pnt

In [20]:
id_list = list(polygon.NAME)

In [21]:
df = pd.DataFrame().reindex_like(points).dropna()

In [23]:
id_list

['Lincoln',
 'Marion',
 'Sherman',
 'Benton',
 'Grant',
 'Baker',
 'Multnomah',
 'Wasco',
 'Douglas',
 'Tillamook',
 'Clatsop',
 'Josephine',
 'Clackamas',
 'Curry',
 'Gilliam',
 'Linn',
 'Deschutes',
 'Jefferson',
 'Malheur',
 'Washington',
 'Wallowa',
 'Lane',
 'Union',
 'Umatilla',
 'Hood River',
 'Yamhill',
 'Wheeler',
 'Coos',
 'Morrow',
 'Polk',
 'Lake',
 'Crook',
 'Jackson',
 'Harney',
 'Columbia',
 'Klamath']

In [25]:
pip_mask = polygon.contains(point.loc[0, 'geometry'])

In [27]:
pip_data = polygon.loc[pip_mask].copy()

In [30]:
pip_data.NAME.values[0]

'Yamhill'

In [31]:
def extract_county_name(point, polygon):
    """
    Extract county name from a point in polygon analysis
    """
    pip_mask = polygon.contains(point.loc[0, 'geometry'])
    if any(pip_mask):
        pip_data = polygon.loc[pip_mask].copy()
    else:
        pip_data = gpd.sjoin_nearest(point, polygon, distance_col='dist')
    cnty = pip_data.NAME.values[0]
    if not any(pip_mask):
        dist = pip_data.dist.values[0]
        print(f'Found {cnty}, about {int(dist+0.5)} degree away')
    return cnty

In [32]:
extract_county_name(point, polygon)

'Yamhill'